<a href="https://colab.research.google.com/github/notnsas/cs229-ml-from-scratch/blob/version-1/cs229_ml_from_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **CS229 ML IMPLEMENTATION**

In [1]:
import numpy as np
import pandas as pd

## **Linear Regression**

In [2]:
# Model Linear Regressi
class LinearRegression:
  def __init__(self, X, y=None, lr=0.01):
    # Params untuk menyimpan parameter dan gradientnya
    self.params = {}

    # inisiasi parameter
    # Menambah col baru dengan semua isinya bernilai 1 ke x
    # yang digunakan agar kita memiliki intercept di rumus linear regresi
    # yang menggunakan vector form
    self.X = np.concatenate((np.ones([np.shape(X)[0], 1]), X),axis=1)
    self.y = y
    # learning rate sebagai konstanta belajar seberapa cepat mengupdate gradient
    self.lr = lr
    # Inisiasi parameter linear regresi dengan menggunakan nilai random
    self.params['W'] = np.random.rand(1, self.X.shape[1])

  def predict(self, X, mode="test"):
    """Untuk Prediksi y Berdasarkan input X menggunakan model linear regresi"""
    if mode == "test":
      # Menambah col baru dengan semua isinya bernilai 1 ke x
      # yang digunakan agar kita memiliki intercept di rumus linear regresi
      # yang menggunakan vector form
      X = np.concatenate((np.ones([np.shape(X)[0], 1]), X),axis=1)
    # Memprediksi y berdasarkan X menggunakan model yang sudah dibuat
    output = np.dot(X, self.params["W"].T)
    return output

  def loss(self, output, y):
    """Menghitung loss"""
    # Menghitung loss menggunakan mean squared error dan dikali 1/2 agar
    # gradientnya nanti gampang dihitung
    return 1/2 * np.mean(np.square(output - y))

  def batch(self, batch_size):
    """Menggunakan mini-batch untuk menghitung loss, output, dan gradien"""
    # Meninisiasi loss dan menshuffle X & y
    loss = 0
    # Menshuffle menggunakan fungsi np.random.permutation() sebagai index
    # lalu menggunakan index tersebut untuk menshuffle X dan y
    indices = np.random.permutation(len(self.y))
    X_shuffle = self.X[indices]
    y_shuffle = self.y[indices]

    # Membagi data menjadi minibatch yg diitung loss nya
    for i in range(0, len(self.y), batch_size):
      end = min(i + batch_size, len(self.y))
      # Membagi data per batch dengan banyak n (training example) sesuai
      # dengan batch_size yang dipilih
      X_batch = X_shuffle[i:end, :]
      y_batch = y_shuffle[i:end]
      # Kalulasi loss per batch
      a = self.calculate(X_batch, y_batch, "train", "gradient_descent")[0]

      # Melakukan running average dari loss sebelumnya agar mendapatkan rata
      # rata loss dari mini batch
      loss = loss + a * (end - i)
    # Return total loss dari loss yang didapatkan menggunakan running average
    # dan dibagi total training (n)
    return loss / len(self.y)


  def calculate(self, X, y, mode="test", solver_type="least_square"):
    """Untuk kalkulasi gradien output dan least square dari model"""
    # Mendapatkan output dari linear regresi
    output = self.predict(X, mode=mode)
    # Bila modenya train
    if mode == "train":
      # Training menggunakan gradient descent
      # Menghitung loss menggunakan output dan y
      loss = self.loss(output, y)

      # Menentukan solver type gradient descent atau metode least square
      if solver_type == "gradient_descent":
        # Menghitung gradient menggunakan rumus dibawah
        self.params['dW'] = np.dot((y[:, np.newaxis] - output).T, X) / len(y)
        # Update parameter berdasarkan gradient dikali konstanta belajar (lr/learning rate)
        self.params['W'] += self.lr * self.params['dW']
      # Bila menggunakan metode least square / metode kuadrat terkecil
      else:
        # Perhitungan menggunakan metode kuadrat terkecil agar mendapatkan
        # Parameter optimal untuk model secara langsung agar meminimalkan
        # loss function
        a = np.linalg.inv(np.dot(X.T, X))
        b = np.dot(X.T, y)
        c = np.dot(a, b)

        # Mengupdate parameter berdasarkan parameter optimal
        self.params['W'] = c[:, np.newaxis].T

    # Return output bila mode test
    else:
      return output

    # Return loss serta output
    return loss, output

  def train(self, solver_type="least_square", epoch=50, batch_size=216):
    """Untuk train model linear regresi"""
    if solver_type == "gradient_descent":
      # Bila solver type gradient descent update parameter dilakukan sampai
      # epoch/iterasi tertentu
      for i in range(epoch):
        # Menghitung loss dari model yang di update parameternya
        loss = self.batch(batch_size)
        print(f"[Epoch {i + 1}] : loss = {loss}")
    else:
      # Menghitung loss yang sudah di update parameternya menggunakan least square/metode kuadrat terkecil
      loss, _ = self.calculate(self.X, self.y, "train", "least_square")
      print(f"loss : {loss}")

In [3]:
# Mengambil data untuk feature X dan label y yaitu 'carat' sebagai kolom yang akan di tebak, lalu menormalisasikanya
X = numeric_data[:, 1:] / np.max(numeric_data[:, 1:],axis=0)
y = numeric_data[:, 0]

# Ngesplit data menjadi train dan testing
X_train, y_train, X_test, y_test = X[:40000, :], y[:40000], X[40000:, :], y[40000:]

# Membuat objek linear regresi dengan data tersebut
model = LinearRegression(X_train, y_train, 0.1)

NameError: name 'numeric_data' is not defined

In [ ]:
# Melatih model linear regresi dan melihat loss dari model tersebut ke data
# Melatih menggunakan metode gradient descent (Karena datanya terlalu banyak)
model.train("gradient_descent", epoch=50, batch_size=512)

In [ ]:
# Melakukan Testing
# Mendapatkan prediksi model berdasarkan X_test yaitu test data
output = model.predict(X_test)
print(f"Hasil output atau prediksi y berdasarkan X adalah : \n{output}")
print(f"\nLabel atau y yang asli sebagai perbandingan : \n{y_test}")

# Menghitung seberapa bagus model menggunakan test data menggunakan Mean Squared Error
mse = np.mean(np.square(output - y_test))
print(f"\n\nMean Squared Error dari Testing data Menggunakan Gradient Descent adalah = {mse}")

Menggunakan metode kuadrat terkecil sebagai metode optimasi

In [ ]:
# Mengambil 15000 data pertama untuk feature X dan 5000 label y yaitu 'carat' sebagai kolom yang akan di tebak
# Data hanya 15000 yang dipakai karena metode kuadrat terkecil tidak dapat dipakai untuk data yang banyak
# Ngesplit data menjadi train dan testing
X_train, y_train, X_test, y_test = X[:15000, :], y[:15000], X[15000:20000, :], y[15000:20000]

# Membuat objek linear regresi dengan data tersebut
model = LinearRegression(X_train, y_train, 0.1)

In [ ]:
# Melatih model linear regresi dan melihat loss dari model tersebut ke data
# Melatih menggunakan metode kuadrat terkecil menggunakan 1000 data
model.train("least_square")

In [ ]:
# Melakukan Testing
# Mendapatkan prediksi model berdasarkan X_test yaitu test data
output = model.predict(X_test)
print(f"Hasil output atau prediksi y berdasarkan X adalah : \n{output}")
print(f"\nLabel atau y yang asli sebagai perbandingan : \n{y_test}")

# Menghitung seberapa bagus model menggunakan test data menggunakan Mean Squared Error
mse = np.mean(np.square(output - y_test))
print(f"\n\nMean Squared Error dari Testing data Menggunakan Metode Kuadrat Terkecil adalah = {mse}")

## **Logistic Regression**

In [ ]:
import numpy as np
class LogisticRegression:
  def __init__(self, X, y=None, lr=0.01):
    # Params untuk menyimpan parameter dan gradientnya
    self.params = {}

    # inisiasi parameter
    # Menambah col baru dengan semua isinya bernilai 1 ke x
    # yang digunakan agar kita memiliki intercept di rumus linear regresi
    # yang menggunakan vector form
    self.X = np.concatenate((np.ones([np.shape(X)[0], 1]), X),axis=1)
    self.y = y
    # learning rate sebagai konstanta belajar seberapa cepat mengupdate gradient
    self.lr = lr
    # Inisiasi parameter linear regresi dengan menggunakan nilai random
    self.params['W'] = np.random.rand(1, self.X.shape[1])

  def predict(self, X, mode="test"):
    """Untuk Prediksi y Berdasarkan input X menggunakan model linear regresi"""
    if mode == "test":
      # Menambah col baru dengan semua isinya bernilai 1 ke x
      # yang digunakan agar kita memiliki intercept di rumus linear regresi
      # yang menggunakan vector form
      X = np.concatenate((np.ones([np.shape(X)[0], 1]), X),axis=1)
      output = self.forward(X, mode)
      output = self.sigmoid(output, mode)
      return np.round(output)
    # Memprediksi y berdasarkan X menggunakan model yang sudah dibuat
    output = self.forward(X, mode)
    output = self.sigmoid(output, mode)
    return output

  def forward(self, X, mode="test"):
    return np.dot(X, self.params["W"].T)

  def sigmoid(self, X, mode="test"):
    return 1 / (1 + np.exp(-X))

  def loss(self, output, y):
    """Menghitung loss"""
    # Menghitung loss menggunakan mean squared error dan dikali 1/2 agar
    # gradientnya nanti gampang dihitung
    # return 1/2 * np.mean(np.square(output - y))
    loss = y * np.log(output) + (1 - y) * np.log((1 - output))
    loss = np.sum(loss)
    # print(loss)
    return loss

  def batch(self, batch_size):
    """Menggunakan mini-batch untuk menghitung loss, output, dan gradien"""
    # Meninisiasi loss dan menshuffle X & y
    loss = 0
    # Menshuffle menggunakan fungsi np.random.permutation() sebagai index
    # lalu menggunakan index tersebut untuk menshuffle X dan y
    indices = np.random.permutation(len(self.y))
    X_shuffle = self.X[indices]
    y_shuffle = self.y[indices]

    # Membagi data menjadi minibatch yg diitung loss nya
    for i in range(0, len(self.y), batch_size):
      end = min(i + batch_size, len(self.y))
      # Membagi data per batch dengan banyak n (training example) sesuai
      # dengan batch_size yang dipilih
      X_batch = X_shuffle[i:end, :]
      y_batch = y_shuffle[i:end]
      # Kalulasi loss per batch
      a = self.calculate(X_batch, y_batch, "train", "gradient_descent")[0]

      # Melakukan running average dari loss sebelumnya agar mendapatkan rata
      # rata loss dari mini batch
      loss = loss + a * (end - i)
    # Return total loss dari loss yang didapatkan menggunakan running average
    # dan dibagi total training (n)
    return loss / len(self.y)


  def calculate(self, X, y, mode="test", solver_type="gradient_descent"):
    """Untuk kalkulasi gradien output dan least square dari model"""
    # Mendapatkan output dari linear regresi
    output = self.predict(X, mode=mode)
    # Bila modenya train
    if mode == "train":
      # Training menggunakan gradient descent
      # Menghitung loss menggunakan output dan y
      loss = self.loss(output, y)

      # Menentukan solver type gradient descent atau metode least square
      if solver_type == "gradient_descent":
        # Menghitung gradient menggunakan rumus dibawah
        self.params['dW'] = np.dot((y[:, np.newaxis] - output).T, X) / len(y)
        # Update parameter berdasarkan gradient dikali konstanta belajar (lr/learning rate)
        self.params['W'] += self.lr * self.params['dW']
      # Bila menggunakan metode least square / metode kuadrat terkecil
      else:
        # Perhitungan menggunakan metode kuadrat terkecil agar mendapatkan
        # Parameter optimal untuk model secara langsung agar meminimalkan
        # loss function
        a = np.linalg.inv(np.dot(X.T, X))
        b = np.dot(X.T, y)
        c = np.dot(a, b)

        # Mengupdate parameter berdasarkan parameter optimal
        self.params['W'] = c[:, np.newaxis].T

    # Return output bila mode test
    else:
      return output

    # Return loss serta output
    return loss, output

  def train(self, solver_type="gradient_descent", epoch=50, batch_size=216):
    """Untuk train model linear regresi"""
    if solver_type == "gradient_descent":
      # Bila solver type gradient descent update parameter dilakukan sampai
      # epoch/iterasi tertentu
      for i in range(epoch):
        # Menghitung loss dari model yang di update parameternya
        loss = self.batch(batch_size)
        print(f"[Epoch {i + 1}] : loss = {loss}")
    else:
      # Menghitung loss yang sudah di update parameternya menggunakan least square/metode kuadrat terkecil
      loss, _ = self.calculate(self.X, self.y, "train", "least_square")
      print(f"loss : {loss}")


In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("fedesoriano/stroke-prediction-dataset")

print("Path to dataset files:", path)

In [ ]:
from os import listdir
from os.path import isfile, join

mypath = '/root/.cache/kagglehub/datasets/fedesoriano/stroke-prediction-dataset/versions/1'
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

In [ ]:
onlyfiles

In [ ]:
import pandas as pd
df = pd.read_csv("/root/.cache/kagglehub/datasets/fedesoriano/stroke-prediction-dataset/versions/1/healthcare-dataset-stroke-data.csv")
df = df.select_dtypes(include='number')
df = df.fillna(df.mean())

In [ ]:
X = df[[i for i in df.select_dtypes(include='number').columns if i!= "stroke"]]
y = df['stroke']


In [ ]:
from sklearn.preprocessing import StandardScaler

# Initialize the scaler
standard_scaler = StandardScaler()

# Fit and transform the data
X = pd.DataFrame(standard_scaler.fit_transform(X), columns=X.columns)
X

In [ ]:
X, y = X.to_numpy(), y.to_numpy()

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y ,
                                   random_state=104,
                                   test_size=0.25,
                                   shuffle=True)

In [ ]:
model = LogisticRegression(X_train, y_train)


In [ ]:
uniques, counts = np.unique(y, return_counts=True)
percentages = dict(zip(uniques, counts * 100 / len(y)))

In [ ]:
percentages

In [ ]:
model.train()

In [ ]:
output = model.predict(X_test)


In [ ]:
acc = np.mean(output == y_test)
acc

In [ ]:
# import the class
from sklearn.linear_model import LogisticRegression

# instantiate the model (using the default parameters)
logreg = LogisticRegression(random_state=16)

# fit the model with data
logreg.fit(X_train, y_train)

y_pred = logreg.predict(X_test)

In [ ]:
np.mean(y_pred == y_test)

In [ ]:
rows = 5
cols = 10

# Create a zero matrix
arr = np.zeros((rows, cols), dtype=int)

# Randomly assign one '1' per row
for i in range(rows):
    j = np.random.randint(cols)  # random column index
    arr[i, j] = 1
arr[1, 1] = 1
arr[2, 2] = 1
print(arr)
import numpy as np

arrs = np.arange(0, 5).reshape(-1, 1)

print(arrs)


In [ ]:
a = np.random.randint(0, 10, (4,4))
a

In [ ]:
np.argmax(a, axis=1)

## **Generalized Linear Model**

The generalized linear model that is used is using multinomial as the label, which the model is called softmax

In [ ]:
class Softmax:
  def __init__(self, X, y=None, lr=0.01):
    # Params untuk menyimpan parameter dan gradientnya
    self.params = {}

    # inisiasi parameter
    # Menambah col baru dengan semua isinya bernilai 1 ke x
    # yang digunakan agar kita memiliki intercept di rumus linear regresi
    # yang menggunakan vector form
    self.X = np.concatenate((np.ones([np.shape(X)[0], 1]), X),axis=1)
    self.y = y
    # learning rate sebagai konstanta belajar seberapa cepat mengupdate gradient
    self.lr = lr
    # Inisiasi parameter linear regresi dengan menggunakan nilai random
    self.params['W'] = np.random.rand(len(np.unique(y)), self.X.shape[1])

  def predict(self, X, mode="test"):
    """Untuk Prediksi y Berdasarkan input X menggunakan model linear regresi"""
    if mode == "test":
      # Menambah col baru dengan semua isinya bernilai 1 ke x
      # yang digunakan agar kita memiliki intercept di rumus linear regresi
      # yang menggunakan vector form
      X = np.concatenate((np.ones([np.shape(X)[0], 1]), X),axis=1)
      output = self.forward(X, mode)
      output = self.softmax(output)
      output = np.argmax(output, axis=1)
      return output
    # Memprediksi y berdasarkan X menggunakan model yang sudah dibuat
    output = self.forward(X, mode)
    output = self.softmax(output)
    return output

  def forward(self, X, mode="test"):
    return np.dot(X, self.params["W"].T)

  def softmax(self, X):
    return np.exp(X) / np.sum(np.exp(X), axis=1, keepdims=True)

  # def loss(self, output, y):
  #   """Menghitung loss"""
  #   y = y.flatten()
  #   num_train = len(y)
  #   # Menghitung loss menggunakan mean squared error dan dikali 1/2 agar
  #   # gradientnya nanti gampang dihitung
  #   # return 1/2 * np.mean(np.square(output - y))
  #   loss = np.log(output[np.arange(num_train), y].reshape(num_train, 1))
  #   loss = np.sum(loss)
  #   # print(loss)
  #   return loss

  def batch(self, batch_size):
    """Menggunakan mini-batch untuk menghitung loss, output, dan gradien"""
    # Meninisiasi loss dan menshuffle X & y
    loss = 0
    # Menshuffle menggunakan fungsi np.random.permutation() sebagai index
    # lalu menggunakan index tersebut untuk menshuffle X dan y
    indices = np.random.permutation(len(self.y))
    X_shuffle = self.X[indices]
    y_shuffle = self.y[indices]

    # Membagi data menjadi minibatch yg diitung loss nya
    for i in range(0, len(self.y), batch_size):
      end = min(i + batch_size, len(self.y))
      # Membagi data per batch dengan banyak n (training example) sesuai
      # dengan batch_size yang dipilih
      X_batch = X_shuffle[i:end, :]
      y_batch = y_shuffle[i:end]
      # Kalulasi loss per batch
      a = self.calculate(X_batch, y_batch, "train", "gradient_descent")[0]

      # Melakukan running average dari loss sebelumnya agar mendapatkan rata
      # rata loss dari mini batch
      loss = loss + a * (end - i)
    # Return total loss dari loss yang didapatkan menggunakan running average
    # dan dibagi total training (n)
    return loss / len(self.y)


  def calculate(self, X, y, mode="test", solver_type="gradient_descent"):
    """Untuk kalkulasi gradien output dan least square dari model"""
    # Mendapatkan output dari linear regresi
    num_train = X.shape[0]
    output = self.predict(X, mode=mode)
    theta = output
    f = theta - np.max(theta, axis=1, keepdims=True)
    scores = np.exp(f) / np.sum(np.exp(f), axis=1, keepdims=True)
    true_scores = scores[np.arange(num_train).flatten(), y.flatten()]
    loss = -np.sum(np.log(true_scores)) / num_train
    # Bila modenya train
    if mode == "train":
      # Training menggunakan gradient descent
      # Menghitung loss menggunakan output dan y


      # Menentukan solver type gradient descent atau metode least square
      if solver_type == "gradient_descent":
        # Menghitung gradient menggunakan rumus dibawah


         # Shift scores to avoid numerical instability
        print(f"X : {X.shape}")
        w_shape = self.params["W"].shape
        print(f"W : {w_shape}")


        print(f"scpres : {scores.shape}")
        print(f"y : {y.shape}")

        print(f"true ; {scores.shape}")

        dx = np.copy(scores)
        dx[np.arange(num_train), y] -= 1
        dx = dx / num_train
        # self.params['dW'] = np.dot((y[:, np.newaxis] - output).T, X) / len(y)
        # Update parameter berdasarkan gradient dikali konstanta belajar (lr/learning rate)
        print(f"X shape : {X.shape}")
        print(f"dx shape : {dx.shape}")
        self.params['W'] -= self.lr * (dx.T @ X)
      # Bila menggunakan metode least square / metode kuadrat terkecil
      else:
        # Perhitungan menggunakan metode kuadrat terkecil agar mendapatkan
        # Parameter optimal untuk model secara langsung agar meminimalkan
        # loss function
        a = np.linalg.inv(np.dot(X.T, X))
        b = np.dot(X.T, y)
        c = np.dot(a, b)

        # Mengupdate parameter berdasarkan parameter optimal
        self.params['W'] = c[:, np.newaxis].T

    # Return output bila mode test
    else:
      return output

    # Return loss serta output
    return loss, output

  def train(self, solver_type="gradient_descent", epoch=50, batch_size=216):
    """Untuk train model linear regresi"""
    if solver_type == "gradient_descent":
      # Bila solver type gradient descent update parameter dilakukan sampai
      # epoch/iterasi tertentu
      for i in range(epoch):
        # Menghitung loss dari model yang di update parameternya
        loss = self.batch(batch_size)
        print(f"[Epoch {i + 1}] : loss = {loss}")
    else:
      # Menghitung loss yanag sudah di update parameternya menggunakan least square/metode kuadrat terkecil
      loss, _ = self.calculate(self.X, self.y, "train", "least_square")
      print(f"loss : {loss}")


In [ ]:
softmax_scratch = Softmax(X_train, y_train)
softmax_scratch.train()

## **import data for multiclass classification**

In [4]:
df_multi = pd.read_csv("https://raw.githubusercontent.com/MachineLearningBCAM/Datasets/refs/heads/main/data/multi_class_datasets/iris.csv")
df_multi.columns = df_multi.iloc[0]
df_multi = df_multi.drop(0)

col_x =  [i for i in df_multi.columns if i != "class"]
df_multi[col_x] = df_multi[col_x].astype(float)
df_multi

,sepal-length-in-cm,sepal-width-in-cm,petal-length-in-cm,petal-width-in-cm,class
1,5.1,3.5,1.4,0.2,Iris-setosa
2,4.9,3.0,1.4,0.2,Iris-setosa
3,4.7,3.2,1.3,0.2,Iris-setosa
4,4.6,3.1,1.5,0.2,Iris-setosa
5,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...
146,6.7,3.0,5.2,2.3,Iris-virginica
147,6.3,2.5,5.0,1.9,Iris-virginica
148,6.5,3.0,5.2,2.0,Iris-virginica
149,6.2,3.4,5.4,2.3,Iris-virginica


In [5]:
df_multi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 1 to 150
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   sepal-length-in-cm  150 non-null    float64
 1   sepal-width-in-cm   150 non-null    float64
 2   petal-length-in-cm  150 non-null    float64
 3   petal-width-in-cm   150 non-null    float64
 4   class               150 non-null    object 
dtypes: float64(4), object(1)
memory usage: 6.0+ KB


In [6]:
# Preprocessing
# Encoding
from sklearn import preprocessing

label_encoder = preprocessing.LabelEncoder()
df_multi['class']= label_encoder.fit_transform(df_multi['class'])

df_multi['class'].unique()

array([0, 1, 2])

In [7]:
X = df_multi[col_x]
y = df_multi['class']

In [8]:
from sklearn.preprocessing import StandardScaler

# Initialize the scaler
standard_scaler = StandardScaler()

# Fit and transform the data
X = pd.DataFrame(standard_scaler.fit_transform(X), columns=X.columns)
X

,sepal-length-in-cm,sepal-width-in-cm,petal-length-in-cm,petal-width-in-cm
0,-0.900681,1.032057,-1.341272,-1.312977
1,-1.143017,-0.124958,-1.341272,-1.312977
2,-1.385353,0.337848,-1.398138,-1.312977
3,-1.506521,0.106445,-1.284407,-1.312977
4,-1.021849,1.263460,-1.341272,-1.312977
...,...,...,...,...
145,1.038005,-0.124958,0.819624,1.447956
146,0.553333,-1.281972,0.705893,0.922064
147,0.795669,-0.124958,0.819624,1.053537
148,0.432165,0.800654,0.933356,1.447956


In [9]:
X, y = X.to_numpy(), y.to_numpy()

In [10]:
# Split data into train and test
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y ,
                                   random_state=104,
                                   test_size=0.25,
                                   shuffle=True)

## **Using the softmax model made from scratch and training it**

In [ ]:
# Initialize
softmax_scratch = Softmax(X_train, y_train, 0.5)
# Train
softmax_scratch.train(epoch=1000)

In [ ]:
output = softmax_scratch.predict(X_test)
acc = np.mean(output == y_test)

print(f"Accuracy of the softmax made by myself is : {acc}")

## **Comparing it with the sklearn softmax regression model**

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Create the softmax regression model
model = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=50)

# Fit the model
model.fit(X_train, y_train)

# Predict on the test set
y_pred = model.predict(X_test)

output = model.predict(X_test)
acc = np.mean(output == y_test)

print(f"Accuracy of the softmax made by sklearn is : {acc}")

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


Accuracy of the softmax made by sklearn is : 1.0


In [15]:
import numpy as np

# First matrix: random integers from 0 to 10
matrix1 = np.random.randint(0, 11, size=(10, 1))

# Second matrix: random 0 or 1
matrix2 = np.random.randint(0, 2, size=(10, 1))

print("Matrix 1:\n", matrix1)
print("\nMatrix 2:\n", matrix2)

Matrix 1:
 [[4]
 [7]
 [0]
 [9]
 [9]
 [5]
 [5]
 [7]
 [7]
 [8]]

Matrix 2:
 [[1]
 [0]
 [0]
 [0]
 [1]
 [1]
 [1]
 [1]
 [1]
 [0]]


In [19]:
# import numpy as np
matrix1[np.where(matrix2 == 0)[0]]

array([[7],
       [0],
       [9],
       [8]])

In [60]:
np.sum(matrix1, axis=1)

array([4, 7, 0, 9, 9, 5, 5, 7, 7, 8])

In [104]:
np.zeros(5)

array([0., 0., 0., 0., 0.])

## **Generative Learning Algorithms**

In [102]:
class GDA:
  def __init__(self, X, y=None, lr=0.01):
    # Params untuk menyimpan parameter dan gradientnya
    self.params = {}

    # inisiasi parameter
    # Menambah col baru dengan semua isinya bernilai 1 ke x
    # yang digunakan agar kita memiliki intercept di rumus linear regresi
    # yang menggunakan vector form
    self.X = np.concatenate((np.ones([np.shape(X)[0], 1]), X),axis=1)
    self.y = y
    # learning rate sebagai konstanta belajar seberapa cepat mengupdate gradient
    self.lr = lr
    # Inisiasi parameter linear regresi dengan menggunakan nilai random
    self.params['W'] = np.random.rand(len(np.unique(y)), self.X.shape[1])

  def predict(self, X, mode="test"):
    """Untuk Prediksi y Berdasarkan input X menggunakan model linear regresi"""
    if mode == "test":
      # Menambah col baru dengan semua isinya bernilai 1 ke x
      # yang digunakan agar kita memiliki intercept di rumus linear regresi
      # yang menggunakan vector form
      print('sblm x')
      X = np.concatenate((np.ones([np.shape(X)[0], 1]), X),axis=1)
      print('hbs x')
      output = self.forward(X)
      print('hbs fwd')
      return output
    # Memprediksi y berdasarkan X menggunakan model yang sudah dibuat
    # output = self.forward(X, mode)
    # output = self.softmax(output)
    return output

  def forward(self, X):
    num_train = X.shape[0]
    output = np.zeros(np.shape(num_train))
    for i in range(num_train):
      print(f"X shape : {X.shape}")
      cov1st = (X[i] - self.params['mu_0']).T
      print(f"cov1st shape : {cov1st.shape}")
      print(f"hbs cov")
      sigma_inv = np.linalg.inv(self.params['sigma'])
      print(f"hbs simga inv")
      cov2nd = (X[i] - self.params['mu_0'])
      print(f"hbs cov2nd")
      tes = cov1st * sigma_inv
      print('hbs yo')
      exp_1st = -1/2 * (X[i] - self.params['mu_0']).T @ np.linalg.inv(self.params['sigma']) @ (X[i] - self.params['mu_0'])
      print('hbs 1st')
      exp_2nd = 1/2 * (X[i] - self.params['mu_1']).T @ np.linalg.inv(self.params['sigma']) @ (X - self.params['mu_1'])
      exp_3rd = np.log(((1 - self.params['phi']) / self.params['mu_1']))
      exp = np.exp(exp_1st + exp_2nd + exp_3rd)
      output = 1 / (1 + exp)
    return output


  def calculate(self, X, y):
    """Untuk kalkulasi gradien output dan least square dari model"""
    # Mendapatkan output dari linear regresi
    # num_train = X.shape[0]
    # output = self.predict(X, mode=mode)
    # theta = output
    # f = theta - np.max(theta, axis=1, keepdims=True)
    # scores = np.exp(f) / np.sum(np.exp(f), axis=1, keepdims=True)
    # true_scores = scores[np.arange(num_train).flatten(), y.flatten()]
    # loss = -np.sum(np.log(true_scores)) / num_train


    # Training menggunakan gradient descent
    # Menghitung loss menggunakan output dan y
    num_train = len(y)


    # Menentukan solver type gradient descent atau metode least square

      # Menghitung gradient menggunakan rumus dibawah
    self.params['phi'] = np.sum(y) / num_train
    list_x_0 = np.where(y == 1)[0]
    self.params['X_0'] = X[list_x_0]
    list_x_1 = np.where(y == 1)[0]
    self.params['X_1'] = X[list_x_1]
    self.params['mu_0'] = np.sum(self.params['X_0'], axis=0) / len(self.params['X_0'])
    self.params['mu_1'] = np.sum(self.params['X_1'], axis=0) / len(self.params['X_1'])
    # self.params['sigma'] = (self.params['X_0'] - self.params['mu_0']) @ (self.params['X_0'] - self.params['mu_0']).T
    d = int(np.shape(X[0])[0])
    sum_cov0 = np.zeros((d, d))

    for i in list_x_0:
      # Reshape so that itll be matrix
      x_i = X[i].reshape(X[i].shape[0], 1)

      # Find cov
      first = (x_i - self.params['mu_0'])
      second = (x_i - self.params['mu_0']).T
      sum_cov0 += first @ second

    sum_cov1 = np.zeros((d, d))
    for i in list_x_1:
      # Reshape so that itll be matrix
      x_i = X[i].reshape(X[i].shape[0], 1)

      # Find cov
      first = (x_i - self.params['mu_1'])
      second = (x_i - self.params['mu_1']).T
      sum_cov1 += first @ second
      # sum_cov1 += (x_i - self.params['mu_1']) @ (x_i - self.params['mu_1']).T

    self.params['sigma'] = (sum_cov0 + sum_cov1 ) / num_train

    # + np.sum((self.params['X_1'] - self.params['mu_1']) @ (self.params['X_1'] - self.params['mu_1']).T)
    # self.params['sigma'] = self.params['sigma'] / num_train


  def train(self):
    """Untuk train model linear regresi"""
    self.calculate(self.X, self.y)


In [103]:
model = GDA(X_train, y_train)
model.train()
model.predict(X_test)

sblm x
hbs x
X shape : (38, 5)
cov1st shape : (5,)
hbs cov
hbs simga inv
hbs cov2nd
hbs yo


ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 5 is different from 38)

In [98]:
model.params['sigma']

array([[ 3.30643757,  0.62863608, -1.38927086,  1.31885021,  0.93613154],
       [ 0.62863608,  2.23436682,  2.19621442,  1.36582028,  1.29405001],
       [-1.38927086,  2.19621442,  5.04811048,  1.05736629,  1.50175659],
       [ 1.31885021,  1.36582028,  1.05736629,  1.29069231,  1.17580369],
       [ 0.93613154,  1.29405001,  1.50175659,  1.17580369,  1.22735376]])

In [99]:
X_test.shape

(38, 4)

In [58]:
model.params['mu_0']

np.float64(-1.7882527424739654)